In [ ]:
#zeroshot

import os
import time
import json
import cv2
import base64
import numpy as np
import pandas as pd
import concurrent.futures

from openai import OpenAI
from google.cloud import storage
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
client = OpenAI(api_key="API_KEY")
# Convert image to base64 string
def convert_image_to_base64(image_path):
    image = cv2.imread(image_path)
    _, buffer = cv2.imencode(".jpg", image)
    return base64.b64encode(buffer).decode("utf-8")

# Analyze a single image using GPT-4 API
def analyze_image(client, image_path):
    base64Image = convert_image_to_base64(image_path)


    prompt_message = (
 "Please provide a prediction of the wound type in the image.\n"
    "Please respond using the format below:\n"
    "{\n"
' “Type": One of the following: “Stab Wound”,  “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions”\n'
    "}\n"
)
    


    prompt_messages = [
        {
            "role": "user",
            "content": [
                prompt_message,
                {"image": base64Image, "resize": 768},
            ],
        }
    ]

    params = {
        "model": "gpt-4o",
        #"model": "gpt-4o-mini-2024-07-18",

      


        
        "messages": prompt_messages,
        "max_tokens": 4096,
        "temperature": 0.0
    }

    try:
        result = client.chat.completions.create(**params)
        text = result.choices[0].message.content
        return text, None
    except Exception as e:
        print(f"Error analyzing image: {e}")
        return None, None

# List images in the local directory and subdirectories
def list_local_images(directory):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                relative_path = os.path.relpath(image_path, directory)
                image_files.append(relative_path.replace(os.sep, '/'))
    return image_files

# Process images in batches
def batch_process_images(client, images, directory, batch_size, output_filename):
    responses = []
    processing_times = []

    def process_image(image):
        image_path = os.path.join(directory, image)
        start_time = time.time()
        response_text, processing_time = analyze_image(client, image_path)
        end_time = time.time()

        if processing_time is None:
            processing_time = end_time - start_time

        response = {image: response_text}
        with open(output_filename, "a") as f:
            f.write(json.dumps(response) + "\n")

        return image, response_text, processing_time

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in range(0, len(images), batch_size):
            print("batch:", i)
            futures = []
            batch = images[i:i + batch_size]
            for image in batch:
                futures.append(executor.submit(process_image, image))

            for future in concurrent.futures.as_completed(futures):
                image_name, response_text, processing_time = future.result()
                responses.append({image_name: response_text})
                processing_times.append((image_name, processing_time))
    return responses, processing_times

def save_responses_to_file(responses, filename):
    with open(filename, 'w') as f:
        json.dump(responses, f, indent=4)

def main_part(directory, model_type, batch_size):
    images = list_local_images(directory)
    print(f"Found {len(images)} images in {directory} and subdirectories.")

    output_dir = "response_baseline/zero shot"
    os.makedirs(output_dir, exist_ok=True)

    jsonl_filename = os.path.join(output_dir, f'responses_{model_type}_images.jsonl')
    json_filename = os.path.join(output_dir, f'responses_{model_type}_images.json')

    responses, processing_times = batch_process_images(client, images, directory, batch_size, output_filename=jsonl_filename)

    for response in responses:
        print(response)

    save_responses_to_file(responses, filename=json_filename)
    print("Responses saved to json")

    df_time = pd.DataFrame(processing_times, columns=['Image Name', 'Processing Time'])
    df_time['Image Name'] = df_time['Image Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

    return jsonl_filename, df_time

if __name__ == "__main__":
    current_dir = os.getcwd()
    directory = os.path.join(current_dir, "dataset")  # Folder with images
    batch_size = 10
    # model_type = 'gpt4o-mini'
    model_type = 'gpt4o'
    filename, df_time = main_part(directory, model_type, batch_size)


In [ ]:
#Chain of thought

import os
import time
import json
import cv2
import base64
import numpy as np
import pandas as pd
import concurrent.futures

from openai import OpenAI
from google.cloud import storage
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
client = OpenAI(api_key="API_KEY")

# Convert image to base64 string
def convert_image_to_base64(image_path):
    image = cv2.imread(image_path)
    _, buffer = cv2.imencode(".jpg", image)
    return base64.b64encode(buffer).decode("utf-8")

# Analyze a single image using GPT-4 API
def analyze_image(client, image_path):
    base64Image = convert_image_to_base64(image_path)



    prompt_message = (
 "You are a medical expert specializing in wound. You will be shown an image from a patient. Your task is to provide a prediction of the wound type in the image from the following seven categories: “Stab Wound”,  “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions”.\n"
    "Please respond using the format below:\n"
    "{\n"
    '  "description": "Provide a concise description of the image, including details such as size, shape, color, depth, exudate, tissue type (e.g., necrotic, granulating), surrounding skin condition, and any other relevant observations. (max 200 words)"\n'
' "reasoning": "Detailed reasoning for why this wound corresponds to the predicted category based on features. (max 100 words)",\n'
' “Type": One of the following: “Stab Wound”,  “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions”\n'
    "}\n"
)
    


    prompt_messages = [
        {
            "role": "user",
            "content": [
                prompt_message,
                {"image": base64Image, "resize": 768},
            ],
        }
    ]

    params = {
        # "model": "gpt-4o",
        "model": "gpt-4o-mini-2024-07-18",
        
        "messages": prompt_messages,
        "max_tokens": 4096,
        "temperature": 0.0
    }

    try:
        result = client.chat.completions.create(**params)
        text = result.choices[0].message.content
        return text, None
    except Exception as e:
        print(f"Error analyzing image: {e}")
        return None, None

# List images in the local directory and subdirectories
def list_local_images(directory):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                relative_path = os.path.relpath(image_path, directory)
                image_files.append(relative_path.replace(os.sep, '/'))
    return image_files

# Process images in batches
def batch_process_images(client, images, directory, batch_size, output_filename):
    responses = []
    processing_times = []

    def process_image(image):
        image_path = os.path.join(directory, image)
        start_time = time.time()
        response_text, processing_time = analyze_image(client, image_path)
        end_time = time.time()

        if processing_time is None:
            processing_time = end_time - start_time

        response = {image: response_text}
        with open(output_filename, "a") as f:
            f.write(json.dumps(response) + "\n")

        return image, response_text, processing_time

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in range(0, len(images), batch_size):
            print("batch:", i)
            futures = []
            batch = images[i:i + batch_size]
            for image in batch:
                futures.append(executor.submit(process_image, image))

            for future in concurrent.futures.as_completed(futures):
                image_name, response_text, processing_time = future.result()
                responses.append({image_name: response_text})
                processing_times.append((image_name, processing_time))
    return responses, processing_times

def save_responses_to_file(responses, filename):
    with open(filename, 'w') as f:
        json.dump(responses, f, indent=4)

def main_part(directory, model_type, batch_size):
    images = list_local_images(directory)
    print(f"Found {len(images)} images in {directory} and subdirectories.")

    output_dir = "response"
    os.makedirs(output_dir, exist_ok=True)

    jsonl_filename = os.path.join(output_dir, f'responses_{model_type}_images_hyp_1.jsonl')
    json_filename = os.path.join(output_dir, f'responses_{model_type}_images_hyp_1.json')

    responses, processing_times = batch_process_images(client, images, directory, batch_size, output_filename=jsonl_filename)

    for response in responses:
        print(response)

    save_responses_to_file(responses, filename=json_filename)
    print("Responses saved to json")

    df_time = pd.DataFrame(processing_times, columns=['Image Name', 'Processing Time'])
    df_time['Image Name'] = df_time['Image Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

    return jsonl_filename, df_time

if __name__ == "__main__":
    current_dir = os.getcwd()
    directory = os.path.join(current_dir, "dataset")  # Folder with images
    batch_size = 10
    model_type = 'gpt4o-mini'
    filename, df_time = main_part(directory, model_type, batch_size)


In [ ]:
#fewshot

import os
import time
import json
import cv2
import base64
import numpy as np
import pandas as pd
import concurrent.futures

from openai import OpenAI
from google.cloud import storage
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
client = OpenAI(api_key="API_KEY")
# Convert image to base64 string
def convert_image_to_base64(image_path):
    image = cv2.imread(image_path)
    _, buffer = cv2.imencode(".jpg", image)
    return base64.b64encode(buffer).decode("utf-8")

# Analyze a single image using GPT-4 API
def analyze_image(client, image_path):
    base64Image = convert_image_to_base64(image_path)



    prompt_message = (f"""
You are a medical expert specializing in wound. You are provided with 3 reference examples of wound analysis. 
Your task is to review the provided examples and provide a prediction of the wound type in the image from the following seven categories: 
“Stab Wound”, “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions".

# Reference Examples:
1. Example 1:
{{
  "description": "The image shows a wound on the skin with irregular, jagged edges. The wound is red and appears to have some depth, with possible minor bleeding. The surrounding skin is slightly red and inflamed, indicating irritation or trauma. There is no significant exudate visible, and the tissue appears raw but not necrotic.",
  "reasoning": "The irregular, jagged edges and the appearance of the wound suggest it is an abrasion. Abrasions are characterized by jagged skin and minor bleeding. The absence of a puncture or burn pattern supports this classification.",
  "Type": "Abrasions"
}}

2. Example 2:
{{
  "description": "The image shows a linear wound with clean, sharp edges, approximately several centimeters in length. The wound is open and appears to be of moderate depth, with visible red tissue and some bleeding. The surrounding skin is slightly reddened but otherwise intact. There is no significant swelling or bruising around the wound. The tissue within the wound appears fresh, with no signs of necrosis or granulation.",
  "reasoning": "The characteristics of the wound, including its linear shape, clean edges, and moderate depth, are indicative of a Cut. Cuts are typically caused by sharp objects and result in clean cuts with bleeding, as seen in the image.",
  "Type": "Cut"
}}

3. Example 3:
{{
  "description": "The image shows a wound with a reddish, irregular shape, surrounded by a purplish discoloration. The wound appears shallow with no significant depth and has a moist appearance. The surrounding skin shows signs of red and purple hues, indicating possible trauma. There is no visible necrotic tissue, and the area does not appear to have significant exudate. The skin around the wound is slightly swollen, suggesting inflammation.",
  "reasoning": "The presence of a reddish, irregular wound with surrounding purplish discoloration and swelling suggests trauma. Blood vessels are damaged under the skin.",
  "Type": "Laceration"
}}

Please think step-by-step and respond using the format below:
        {{
        "description": "Provide a concise description of the image, including details such as size, shape, color, depth, exudate, tissue type (e.g., necrotic, granulating), surrounding skin condition, and any other relevant observations. (max 200 words)",
        "reasoning": "Detailed reasoning for why this wound corresponds to the predicted category based on features. (max 100 words)",
        "Type": One of the following: “Stab Wound”, “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions”
        }}
"""
)
    


    prompt_messages = [
        {
            "role": "user",
            "content": [
                prompt_message,
                {"image": base64Image, "resize": 768},
            ],
        }
    ]

    params = {
        "model": "gpt-4o",
        #"model": "gpt-4o-mini-2024-07-18",

      


        
        "messages": prompt_messages,
        "max_tokens": 4096,
        "temperature": 0.0
    }

    try:
        result = client.chat.completions.create(**params)
        text = result.choices[0].message.content
        return text, None
    except Exception as e:
        print(f"Error analyzing image: {e}")
        return None, None

# List images in the local directory and subdirectories
def list_local_images(directory):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                relative_path = os.path.relpath(image_path, directory)
                image_files.append(relative_path.replace(os.sep, '/'))
    return image_files

# Process images in batches
def batch_process_images(client, images, directory, batch_size, output_filename):
    responses = []
    processing_times = []

    def process_image(image):
        image_path = os.path.join(directory, image)
        start_time = time.time()
        response_text, processing_time = analyze_image(client, image_path)
        end_time = time.time()

        if processing_time is None:
            processing_time = end_time - start_time

        response = {image: response_text}
        with open(output_filename, "a") as f:
            f.write(json.dumps(response) + "\n")

        return image, response_text, processing_time

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in range(0, len(images), batch_size):
            print("batch:", i)
            futures = []
            batch = images[i:i + batch_size]
            for image in batch:
                futures.append(executor.submit(process_image, image))

            for future in concurrent.futures.as_completed(futures):
                image_name, response_text, processing_time = future.result()
                responses.append({image_name: response_text})
                processing_times.append((image_name, processing_time))
    return responses, processing_times

def save_responses_to_file(responses, filename):
    with open(filename, 'w') as f:
        json.dump(responses, f, indent=4)

def main_part(directory, model_type, batch_size):
    images = list_local_images(directory)
    print(f"Found {len(images)} images in {directory} and subdirectories.")

    output_dir = "response_baseline/few shot"
    os.makedirs(output_dir, exist_ok=True)

    jsonl_filename = os.path.join(output_dir, f'responses_{model_type}_images.jsonl')
    json_filename = os.path.join(output_dir, f'responses_{model_type}_images.json')

    responses, processing_times = batch_process_images(client, images, directory, batch_size, output_filename=jsonl_filename)

    for response in responses:
        print(response)

    save_responses_to_file(responses, filename=json_filename)
    print("Responses saved to json")

    df_time = pd.DataFrame(processing_times, columns=['Image Name', 'Processing Time'])
    df_time['Image Name'] = df_time['Image Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

    return jsonl_filename, df_time

if __name__ == "__main__":
    current_dir = os.getcwd()
    directory = os.path.join(current_dir, "dataset")  # Folder with images
    batch_size = 10
    # model_type = 'gpt4o-mini'
    model_type = 'gpt4o'
    filename, df_time = main_part(directory, model_type, batch_size)


In [ ]:
#ICL

import os
import time
import json
import cv2
import base64
import numpy as np
import pandas as pd
import concurrent.futures

from openai import OpenAI
from google.cloud import storage
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
client = OpenAI(api_key="API_KEY")
# Convert image to base64 string
def convert_image_to_base64(image_path):
    image = cv2.imread(image_path)
    _, buffer = cv2.imencode(".jpg", image)
    return base64.b64encode(buffer).decode("utf-8")

# Analyze a single image using GPT-4 API
def analyze_image(client, image_path):
    base64Image = convert_image_to_base64(image_path)


    prompt_message = (f"""
You are a medical expert specializing in wound. You are provided with a set of rules for wound prediction.
Your task is to review the provided rules and provide a prediction of the wound type in the image from the following seven categories: 
“Stab Wound”, “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions".

The rules provided for wound prediction are: 
"Stab Wound: Puncture-like wound with narrow entry, often deep and sharply defined; minimal surface tearing.",
"Laceration: Irregular, jagged wound with torn or split skin, often caused by blunt trauma.",
"Ingrown Nails: Redness, swelling, and possible pus near toenail or fingernail edges, often with nail edge embedded in skin.",
"Cut: Clean, linear break in the skin, usually shallow and caused by sharp objects like knives.",
"Burns: Reddened, blistered, or charred skin with varying depth; may appear dry, leathery, or moist depending on severity.",
"Bruises: Discoloration under intact skin (blue, purple, green, or yellow), typically caused by blunt force trauma without open wound.",
"Abrasions: Superficial scraping of the skin surface, often with visible raw or red areas and minimal bleeding."

Please think step-by-step and respond using the format below:
        {{
        "description": "Provide a concise description of the image, including details such as size, shape, color, depth, exudate, tissue type (e.g., necrotic, granulating), surrounding skin condition, and any other relevant observations. (max 200 words)",
        "reasoning": "Detailed reasoning for why this wound corresponds to the predicted category based on features. (max 100 words)",
        "Type": One of the following: “Stab Wound”, “Laceration”, “Ingrown Nails”, “Cut”, “Burns”, “Bruises”, “Abrasions”
        }}
"""
)
    


    prompt_messages = [
        {
            "role": "user",
            "content": [
                prompt_message,
                {"image": base64Image, "resize": 768},
            ],
        }
    ]

    params = {
        #"model": "gpt-4o",
        "model": "gpt-4o-mini-2024-07-18",

      


        
        "messages": prompt_messages,
        "max_tokens": 4096,
        "temperature": 0.0
    }

    try:
        result = client.chat.completions.create(**params)
        text = result.choices[0].message.content
        return text, None
    except Exception as e:
        print(f"Error analyzing image: {e}")
        return None, None

# List images in the local directory and subdirectories
def list_local_images(directory):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                relative_path = os.path.relpath(image_path, directory)
                image_files.append(relative_path.replace(os.sep, '/'))
    return image_files

# Process images in batches
def batch_process_images(client, images, directory, batch_size, output_filename):
    responses = []
    processing_times = []

    def process_image(image):
        image_path = os.path.join(directory, image)
        start_time = time.time()
        response_text, processing_time = analyze_image(client, image_path)
        end_time = time.time()

        if processing_time is None:
            processing_time = end_time - start_time

        response = {image: response_text}
        with open(output_filename, "a") as f:
            f.write(json.dumps(response) + "\n")

        return image, response_text, processing_time

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in range(0, len(images), batch_size):
            print("batch:", i)
            futures = []
            batch = images[i:i + batch_size]
            for image in batch:
                futures.append(executor.submit(process_image, image))

            for future in concurrent.futures.as_completed(futures):
                image_name, response_text, processing_time = future.result()
                responses.append({image_name: response_text})
                processing_times.append((image_name, processing_time))
    return responses, processing_times

def save_responses_to_file(responses, filename):
    with open(filename, 'w') as f:
        json.dump(responses, f, indent=4)

def main_part(directory, model_type, batch_size):
    images = list_local_images(directory)
    print(f"Found {len(images)} images in {directory} and subdirectories.")

    output_dir = "response_baseline/ICL"
    os.makedirs(output_dir, exist_ok=True)

    jsonl_filename = os.path.join(output_dir, f'responses_{model_type}_images.jsonl')
    json_filename = os.path.join(output_dir, f'responses_{model_type}_images.json')

    responses, processing_times = batch_process_images(client, images, directory, batch_size, output_filename=jsonl_filename)

    for response in responses:
        print(response)

    save_responses_to_file(responses, filename=json_filename)
    print("Responses saved to json")

    df_time = pd.DataFrame(processing_times, columns=['Image Name', 'Processing Time'])
    df_time['Image Name'] = df_time['Image Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

    return jsonl_filename, df_time

if __name__ == "__main__":
    current_dir = os.getcwd()
    directory = os.path.join(current_dir, "dataset")  # Folder with images
    batch_size = 10
    model_type = 'gpt4o-mini'
    #model_type = 'gpt4o'
    filename, df_time = main_part(directory, model_type, batch_size)
